## TODO 
- maybe some better model for embedding extraction
- better prompt for the chatbot 
- somehow test the implementation
- Adding citations to score the papers
- somehow separate the user query and searching for papers on arxiv

## Generate a response by incoroprating the retrieved papers with a chatbot

## Larger model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig, AutoConfig
import torch
import dotenv

# Load a chat-capable model
LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
device = f"cuda:{torch.cuda.current_device()}"

tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=LLM_MODEL,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 8-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # loading in 4 bit
    bnb_4bit_quant_type="nf4", # quantization type
    bnb_4bit_use_double_quant=True, # nested quantization
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_config = AutoConfig.from_pretrained(
    pretrained_model_name_or_path=LLM_MODEL,
)
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=LLM_MODEL,
    config=model_config,
    quantization_config=bnb_config, # we introduce the bnb config here.
    device_map="auto",
)
model.eval()

c:\Users\sebas\one\OneDrive\grive\faks\masters\y1\2nd semester\NLP\ul-fri-nlp-course-project-2024-2025-optimus-prime\.venv\lib\site-packages\accelerate\utils\modeling.py:1569: UserWarning: Current model requires 32.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

: 

In [ ]:
from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate

# Define the Hugging Face pipeline for text generation
generate_text = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    max_new_tokens=8192,
    repetition_penalty=1.1,
)

# Wrap the Hugging Face pipeline into LangChain's LLM
llm = HuggingFacePipeline(pipeline=generate_text)

template = """
You are a helpful AI QA assistant, for answering queries about research methods.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Question: {question}
Answer:"""
prompt = PromptTemplate.from_template(template)

Device set to use cuda:0
C:\Users\sebas\AppData\Local\Temp\ipykernel_3356\2378073949.py:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generate_text)


In [ ]:
# an example of something that works without rag
chain = prompt | llm
question = "Which paper introduced the transformer architecture"
print(chain.invoke({"question": question}))


You are a helpful AI QA assistant, for answering querries about research methods.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Question: Which paper introduced the transformer architecture
Answer: The transformer architecture was introduced in the paper "Attention is All You Need" by Vaswani et al., published in 2017.


In [ ]:
# A newer one 
question = ""
print(chain.invoke({"question": question}))


You are a helpful AI QA assistant, for answering querries about research methods.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Question: 
Answer: I'd be happy to help answer any questions you have about research methods! However, your question is not specific enough for me to provide a clear answer. Could you please specify which research method or methods you are inquiring about? Some common research methods include surveys, experiments, case studies, and literature reviews. Once I have more information, I can provide a more accurate response.


In [ ]:
question = "What is the latest training data you have been trained on?"
print(chain.invoke({"question": question}))


You are a helpful AI QA assistant, for answering querries about research methods.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Question: What is the latest training data you have been trained on?
Answer: I don't have the ability to be trained on data or to have a specific training dataset. I am a text-based AI model and do not rely on data to generate responses. I am designed to process and understand natural language input and provide relevant information based on that input and my programming.


In [ ]:
question = "What can you tell me about the paper Instruct-ReID: A Multi-purpose Person Re-identification Task with Instructions"
print(chain.invoke({"question": question}))


You are a helpful AI QA assistant, for answering querries about research methods.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Question: What can you tell me about the paper Instruct-ReID: A Multi-purpose Person Re-identification Task with Instructions
Answer: "Instruct-ReID" is a paper published in the IEEE Transactions on Pattern Analysis and Machine Intelligence journal. The authors propose a new multi-purpose person re-identification (ReID) task, which involves not only identifying the same person across different cameras but also understanding and following given instructions related to the person. This task is designed to evaluate the ability of models to reason about context and follow instructions, in addition to their ability to recognize people. The paper provides a detailed description of the dataset and evaluation protocol used for this task. If you need more specific information, please let me know.


## Get the papers based on the user query

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer 
import arxiv 

# Load the sentence transformer model
# TODO maybe some better model for embedding extraction, maybe we could fine tune?
# TODO maybe somehow add citations
# model_embeddings = SentenceTransformer('all-mpnet-base-v2')  # For embeddin extraction
model_embeddings = SentenceTransformer('allenai-specter') # It can be used to map the titles & abstracts of scientific publications to a vector space such that similar papers are close.

# Define your query
user_query = "Toward Efficient Exploration by Large Language Model Agents"  # arxiv actually finds this one

# Get the embedding for the query
query_embedding = model_embeddings.encode([user_query])

search = arxiv.Search(
    query=user_query,
    max_results=50,
    sort_by=arxiv.SortCriterion.Relevance,
    sort_order=arxiv.SortOrder.Descending
)

client = arxiv.Client()
results = list(client.results(search))

# Extract summaries and titles
papers = []
summaries = []
for result in results:
    title = result.title
    authors = ', '.join([author.name for author in result.authors])
    summary = result.summary
    url = f"https://arxiv.org/abs/{result.entry_id.split('/')[-1]}"
    papers.append({
        "title": title,
        "authors": authors,
        "summary": summary,
        "url": url
    })
    summaries.append(summary)

# Encode all summaries
summary_embeddings = model_embeddings.encode(summaries)

# Compute cosine similarities
similarities = cosine_similarity(query_embedding, summary_embeddings)[0]

for i, paper in enumerate(papers):
    paper["similarity"] = similarities[i]


top_papers = sorted(papers, key=lambda x: x["similarity"], reverse=True)[:5] # top 5

# Print top 5 similar papers
for i, paper in enumerate(top_papers, 1):
    print(f"Rank #{i}")
    print(f"Title: {paper['title']}")
    print(f"Authors: {paper['authors']}")
    print(f"Summary: {paper['summary']}")
    print(f"Similarity: {paper['similarity']:.4f}")
    print(f"URL: {paper['url']}")
    print("-" * 80)


c:\Users\sebas\one\OneDrive\grive\faks\masters\y1\2nd semester\NLP\ul-fri-nlp-course-project-2024-2025-optimus-prime\.venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sebas\.cache\huggingface\hub\models--sentence-transformers--allenai-specter. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Stor

Rank #1
Title: Position: Foundation Agents as the Paradigm Shift for Decision Making
Authors: Xiaoqian Liu, Xingzhou Lou, Jianbin Jiao, Junge Zhang
Summary: Decision making demands intricate interplay between perception, memory, and
reasoning to discern optimal policies. Conventional approaches to decision
making face challenges related to low sample efficiency and poor
generalization. In contrast, foundation models in language and vision have
showcased rapid adaptation to diverse new tasks. Therefore, we advocate for the
construction of foundation agents as a transformative shift in the learning
paradigm of agents. This proposal is underpinned by the formulation of
foundation agents with their fundamental characteristics and challenges
motivated by the success of large language models (LLMs). Moreover, we specify
the roadmap of foundation agents from large interactive data collection or
generation, to self-supervised pretraining and adaptation, and knowledge and
value alignment with L

## Combine the retrieved papers and the generation model

In [ ]:
from langchain.chains import LLMChain

# Combine summaries into a context string
context = "\n\n".join(
    f"Title: {paper['title']}\nSummary: {paper['summary']}" for paper in top_papers
)


PROMPT_TEMPLATE = """
You are a helpful AI QA assistant, for answering querries about research methods.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

```
{context}
```

### Question:
{question}

### Answer:
"""

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=PROMPT_TEMPLATE.strip(),
)

# Define the Hugging Face pipeline for text generation
generate_text = pipeline(
    task="text-generation",
    model=model,  # Replace with your model
    tokenizer=tokenizer,  # Replace with your tokenizer
    return_full_text=True,
    max_new_tokens=8192,
    repetition_penalty=1.1,
)

# Wrap the Hugging Face pipeline into LangChain's LLM
llm = HuggingFacePipeline(pipeline=generate_text)

# Create the LLMChain with the prompt template and the LLM
qa_chain = LLMChain(prompt=prompt_template, llm=llm)

# Ask the model a question and get the answer
question = user_query # TODO maybe change this, to make it different than the search (or change the search)
response = qa_chain.run({"context": context, "question": question})

# Print the response
print("Answer:", response)

Device set to use cuda:0
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GP

Answer: You are a helpful AI QA assistant, for answering querries about research methods.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

```
Title: Instruct-ReID++: Towards Universal Purpose Instruction-Guided Person Re-identification
Summary: Human intelligence can retrieve any person according to both visual and
language descriptions. However, the current computer vision community studies
specific person re-identification (ReID) tasks in different scenarios
separately, which limits the applications in the real world. This paper strives
to resolve this problem by proposing a novel instruct-ReID task that requires
the model to retrieve images according to the given image or language
instructions. Instruct-ReID is the first exploration of a general ReID setting,
where existing 6 ReID tasks can be viewed as special cases by assigning
different instructions. To facilitate research in this new instruct-ReID task,
we propose a large-scale OmniRe

## Simpler model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

rag = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Combine summaries into a context string, but make sure it's within the token limit
context = "\n\n".join(
    f"Title: {paper['title']}\nSummary: {paper['summary']}" for paper in top_papers
)

# Encode the context and check its length
input_ids = tokenizer.encode(context, return_tensors="pt")
max_length = 1700  # Adjust this based on your model's max token length

# Truncate if necessary to fit within the max token limit
if input_ids.shape[1] > max_length:
    input_ids = input_ids[:, :max_length]


# Prepare the prompt, ensuring it stays within the token limit
prompt = f"""Here are some research papers:

{context[:max_length]}  # Only include a truncated context if necessary

Use the above research paper summaries to answer the following question:

Question: {user_query}
Answer:"""

# Generate the answer using the same prompt
output = rag(prompt, max_new_tokens=300)

# Provide the generated answer along with the papers
print("Research Papers and Generated Answer:")
print(f"Research Papers:\n{context[:max_length]}")  # Display truncated context
print(f"Generated Answer:\n{output[0]['generated_text']}")


c:\Users\sebas\one\OneDrive\grive\faks\masters\y1\2nd semester\NLP\ul-fri-nlp-course-project-2024-2025-optimus-prime\.venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sebas\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for

Research Papers and Generated Answer:
Research Papers:
Title: Neuromodulation Gated Transformer
Summary: We introduce a novel architecture, the Neuromodulation Gated Transformer
(NGT), which is a simple implementation of neuromodulation in transformers via
a multiplicative effect. We compare it to baselines and show that it results in
the best average performance on the SuperGLUE benchmark validation sets.

Title: Interpretation of the Transformer and Improvement of the Extractor
Summary: It has been over six years since the Transformer architecture was put
forward. Surprisingly, the vanilla Transformer architecture is still widely
used today. One reason is that the lack of deep understanding and comprehensive
interpretation of the Transformer architecture makes it more challenging to
improve the Transformer architecture. In this paper, we first interpret the
Transformer architecture comprehensively in plain words based on our
understanding and experiences. The interpretations are furt